In [ ]:
# # 用於連接 PostgreSQL
# !pip install psycopg2-binary
# !pip install python-dotenv
# # 用於載入 CLIP 模型 (HuggingFace 推薦)
# !pip install sentence-transformers

# # 用於開啟圖片
# !pip install Pillow
#!pip install tqdm
# # 降維度的套件與可視化的套件
#!pip install pandas scikit-learn plotly

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 8.5 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 17.7 MB/s  0:00:00 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [pandas]2m4/5 [pandas]s]


# 下載圖片向量並匯入資料庫中

import json
import os
import requests
import time
import psycopg2
from PIL import Image
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# [NEW] 匯入網路重試工具
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
# [NEW] 匯入進度條工具
from tqdm import tqdm 

load_dotenv()

# --- 1. 檔案與模型設定 ---
LDJSON_FILE_PATH = 'marketing_sample_for_amazon_com-amazon_fashion_products__20200201_20200430__30k_data.ldjson'
IMG_DIR = "img"
ERROR_LOG_FILE = "download_errors.txt"
MODEL_NAME = 'clip-ViT-L-14'

# [修正 1]：設定 MAX_RECORDS 為 None，才能跑完整個檔案
MAX_RECORDS = None # 設為 None 來處理 30k 筆資料

# --- 2. 資料庫連線設定 ---
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")

# --- 3. [修正 2]：設定網路重試機制 ---
# 為了應對您「網路不太好」的問題
session = requests.Session()
retry_strategy = Retry(
    total=5,  # 總共重試 5 次
    status_forcelist=[429, 500, 502, 503, 504], # 遇到這些伺服器錯誤時重試
    backoff_factor=1 # 重試間隔 (1s, 2s, 4s, 8s, 16s)
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)
print("網路重試機制已設定。")
# --- 結束設定 ---

# 載入 AI 模型
print(f"正在載入 AI 模型 '{MODEL_NAME}'...")
try:
    model = SentenceTransformer(MODEL_NAME)
    print("AI 模型載入成功。")
except Exception as e:
    print(f"致命錯誤：無法載入 AI 模型。錯誤：{e}")
    exit()

# 連接到 PostgreSQL
try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = conn.cursor()
    print(f"成功連接到 PostgreSQL 資料庫 '{DB_NAME}'。")
except psycopg2.Error as e:
    print(f"致命錯誤：無法連接到資料庫。錯誤：{e}")
    exit()

# 建立 'img' 資料夾
if not os.path.exists(IMG_DIR):
    os.makedirs(IMG_DIR)

# [修正 4]：加入 tqdm 進度條
# 為了能顯示進度，我們先快速計算總行數
try:
    print("正在計算總資料筆數...")
    with open(LDJSON_FILE_PATH, 'r', encoding='utf-8') as f_count:
        total_lines = sum(1 for _ in f_count)
    print(f"總共 {total_lines} 筆資料。")
except FileNotFoundError:
    print(f"致命錯誤：找不到檔案 '{LDJSON_FILE_PATH}'")
    exit()

print(f"--- 開始處理資料 (下載、向量化、寫入資料庫) ---")

download_count = 0
skip_count = 0
error_count = 0
db_insert_count = 0
error_list = []

try:
    with open(LDJSON_FILE_PATH, 'r', encoding='utf-8') as f:
        # [修正 4] 使用 tqdm 包裹 enumerate
        pbar = tqdm(enumerate(f), total=total_lines, desc="處理中", unit=" 筆")
        
        for i, line in pbar:
            
            if MAX_RECORDS is not None and (download_count + skip_count) >= MAX_RECORDS:
                print(f"\n已達到 {MAX_RECORDS} 筆的處理上限，停止腳本。")
                break
            
            data = None
            try:
                data = json.loads(line)
                
                # --- 1. 獲取欄位 ---
                uniq_id = data.get('uniq_id')
                product_name = data.get('product_name')
                image_url = data.get('medium', '').split('|')[0]
                brand = data.get('brand')
                sales_price_str = data.get('sales_price')
                sales_price = float(sales_price_str) if sales_price_str and sales_price_str.replace('.', '', 1).isdigit() else None
                rating_str = data.get('rating', '').split(' ')[0]
                rating = float(rating_str) if rating_str and rating_str.replace('.', '', 1).isdigit() else None
                best_seller_tag = data.get('best_seller_tag__y_or_n', 'N') == 'Y'
                browsenode = data.get('browsenode')
                meta_keywords = data.get('meta_keywords')

                if not uniq_id or not image_url:
                    error_count += 1
                    error_list.append(f"Line {i+1}: Missing uniq_id or medium image_url.")
                    continue

                # --- 2. 圖片處理 ---
                filename_base = uniq_id[-10:]
                filename_jpg = f"{filename_base}.jpg"
                save_path = os.path.join(IMG_DIR, filename_jpg)

                image_to_process = None

                if os.path.exists(save_path):
                    skip_count += 1
                    image_to_process = save_path
                else:
                    # [修正 3] 增加網路超時
                    # [修正 2] 使用 session.get 進行重試
                    response = session.get(image_url, timeout=30) # 超時延長到 30 秒
                    
                    if response.status_code == 200:
                        with open(save_path, 'wb') as img_file:
                            img_file.write(response.content)
                        download_count += 1
                        image_to_process = save_path
                    else:
                        error_count += 1
                        error_list.append(f"{uniq_id}: HTTP Error {response.status_code} for URL {image_url}")
                        continue

                # --- 3. 產生向量 ---
                try:
                    pil_image = Image.open(image_to_process)
                    embedding = model.encode(pil_image)
                except Exception as e:
                    error_count += 1
                    error_list.append(f"{uniq_id}: Vectorization Error ({e})")
                    continue

                # --- 4. 寫入資料庫 ---
                try:
                    sql = """
                    INSERT INTO products (
                        id, product_name, image_url, brand, sales_price, 
                        rating, best_seller_tag, browsenode, meta_keywords, embedding
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (id) DO NOTHING; 
                    """
                    
                    data_tuple = (
                        uniq_id, product_name, image_url, brand, sales_price,
                        rating, best_seller_tag, browsenode, meta_keywords, 
                        embedding.tolist() #
                    )
                    
                    cursor.execute(sql, data_tuple)
                    db_insert_count += 1
                    
                    if db_insert_count % 100 == 0:
                        conn.commit()
                        # [修正 4] 更新 tqdm 進度條的後綴
                        pbar.set_postfix_str(f"Inserted: {db_insert_count}, Skipped: {skip_count}, Errors: {error_count}")
                        
                except psycopg2.Error as e:
                    conn.rollback()
                    error_count += 1
                    error_list.append(f"{uniq_id}: DB Insert Error ({e})")
                
                # [修正 3] 縮短 sleep 時間，因為重試機制會處理網路延遲
                time.sleep(0.01)

            # (保留錯誤處理)
            except json.JSONDecodeError:
                error_count += 1
                error_list.append(f"Line {i+1}: JSON Decode Error. Content: {line[:50]}...")
            except requests.exceptions.RequestException as e:
                uniq_id_log = data['uniq_id'] if (data and "uniq_id" in data) else f"Line {i+1}"
                error_count += 1
                error_list.append(f"{uniq_id_log}: Network Error ({e})")
            except KeyError as e:
                uniq_id_log = data['uniq_id'] if (data and "uniq_id" in data) else f"Line {i+1}"
                error_count += 1
                error_list.append(f"{uniq_id_log}: Missing Key {e}")
            except Exception as e:
                uniq_id_log = data['uniq_id'] if (data and "uniq_id" in data) else f"Line {i+1}"
                error_count += 1
                error_list.append(f"{uniq_id_log}: Unknown Error ({e})")

except FileNotFoundError:
    print(f"致命錯誤：找不到檔案 '{LDJSON_FILE_PATH}'")
except psycopg2.Error as e:
    print(f"致命錯誤：資料庫連線中斷。錯誤：{e}")
except Exception as e:
    print(f"腳本因未預期錯誤而終止： {e}")

finally:
    if 'pbar' in locals():
        pbar.close() # [修正 4] 關閉進度條

    try:
        if 'conn' in locals() and conn:
            conn.commit()
            print("提交最後的資料庫變更...")
            cursor.close()
            conn.close()
            print("資料庫連線已關閉。")
    except Exception as e:
        print(f"錯誤：關閉資料庫連線時發生錯誤。{e}")

    print("\n" + "-" * 40)
    print("階段 1.3 腳本執行完畢。")
    print(f"  成功下載 (新)：{download_count} 張圖片")
    print(f"  已存在/跳過：{skip_count} 筆資料")
    print(f"  成功寫入資料庫：{db_insert_count} 筆紀錄")
    print(f"  錯誤/失敗：{error_count} 筆")
    print(f"  圖片已儲存於 '{IMG_DIR}' 資料夾中。")
    
    if error_count > 0:
        print(f"\n正在將 {error_count} 筆錯誤紀錄寫入 '{ERROR_LOG_FILE}'...")
        try:
            # [修正 5] 將 'w' (覆寫) 改為 'a' (附加)
            with open(ERROR_LOG_FILE, 'a', encoding='utf-8') as err_f:
                err_f.write(f"\n--- Log Entry: {time.ctime()} ---\n") # [NEW] 加入時間戳
                err_f.write(f"此次執行總共 {error_count} 筆新錯誤。\n")
                err_f.write("-" * 30 + "\n")
                for error_entry in error_list:
                    err_f.write(f"{error_entry}\n")
            print(f"錯誤日誌已成功附加到 '{ERROR_LOG_FILE}'。")
        except Exception as e:
            print(f"錯誤：無法將日誌寫入檔案 '{ERROR_LOG_FILE}'。錯誤：{e}")

# 降維度

In [3]:
import psycopg2
from psycopg2 import sql
import pandas as pd # 用於資料處理
import numpy as np
from sklearn.decomposition import PCA # 降維工具 1
from sklearn.manifold import TSNE  # 降維工具 2
import plotly.graph_objects as go # 3D 繪圖工具
import time
import os
from dotenv import load_dotenv
import json
load_dotenv()

# --- 2. 資料庫連線設定 ---
DB_NAME = os.environ.get("DB_NAME")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")

# 2. 抽樣數量
# 執行 t-SNE 非常耗時 (O(N^2))
# 您的資料集約 3 萬筆，建議先從 5000 筆開始測試
# 如果 5000 筆跑得很快 (例如 1 分鐘內)，您可以再調高
SAMPLE_SIZE = 5000 

# 3. 輸出檔案名稱
OUTPUT_HTML_FILE = "vector_3d_plot.html"
# --- 結束設定 ---

def fetch_data_from_db():
    """從 PostgreSQL 讀取資料並載入到 Pandas DataFrame"""
    print(f"正在連線至資料庫 '{DB_NAME}'...")
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cursor = conn.cursor()

        print(f"連線成功。正在讀取 {SAMPLE_SIZE} 筆資料...")
        
        # 我們讀取 CBO (盾) 和 AI (矛) 需要的所有關鍵欄位
        query = sql.SQL("""
            SELECT id, product_name, brand, sales_price, rating, embedding 
            FROM products 
            ORDER BY random() -- 隨機抽樣
            LIMIT {limit};
        """).format(limit=sql.Literal(SAMPLE_SIZE))

        cursor.execute(query) # 步驟 1: 執行查詢
        results = cursor.fetchall() # 步驟 2: 獲取所有結果
        
        # 步驟 3: 獲取欄位名稱
        colnames = [desc[0] for desc in cursor.description]
        
        # 步驟 4: 建立 DataFrame
        df = pd.DataFrame(results, columns=colnames)
        
        print(f"成功讀取 {len(df)} 筆資料。")
        
        if df['embedding'].isnull().any():
            print("警告：資料中有 'embedding' 欄位為空，將移除這些資料。")
            df = df.dropna(subset=['embedding'])
            

        # 處理 sales_price 欄位中的空值 (NULL/None/NaN)
        # 這是導致 TypeError 的主要原因
        if df['sales_price'].isnull().any():
            print("警告：資料中有 'sales_price' 欄位為空，將填補為 0.0。")
            # .fillna(0.0) 會將所有 None/NaN 替換為 0.0
            df['sales_price'] = df['sales_price'].fillna(0.0)

        # [改進] 我們也應該處理 brand 欄位中的空值
        if df['brand'].isnull().any():
            print("警告：資料中有 'brand' 欄位為空，將填補為 'N/A'。")
            df['brand'] = df['brand'].fillna('N/A')
        # --- [BUG 修正完畢] ---
            
        return df


    except Exception as e:
        print(f"讀取資料庫時發生錯誤：{e}")
        return None
    finally:
        if conn:
            conn.close()

def reduce_dimensions(df):
    """執行 PCA -> t-SNE 降維 (768D -> 50D -> 3D)"""
    if df.empty:
        print("沒有資料可供降維。")
        return None
        
    # 【BUG 2 修正】 在這裡，df['embedding'] 仍然是「字串」
    print("正在將 768 維向量 (字串) 轉換為 Python 列表...")
    
    try:
        # 我們使用 .apply(json.loads) 來將每一行的「字串」(例如 '[0.1, 0.2, ...]')
        # 轉換成一個「Python 列表」 (例如 [0.1, 0.2, ...])
        embedding_lists = df['embedding'].apply(json.loads)
    except json.JSONDecodeError as e:
        print(f"錯誤：解析 'embedding' 欄位時發生 JSON 錯誤。")
        print(f"錯誤的資料範例 (類型: {type(df['embedding'].values[0])})：{df['embedding'].values[0][:50]}...")
        print(f"詳細錯誤：{e}")
        return None
    print("正在將 Python 列表轉換為 NumPy 矩陣...")
    # 現在 embedding_lists 是一個包含「列表」的 Series，
    # np.stack 可以正確地將它們堆疊成一個 (N, 768) 的矩陣
    vectors_768d = np.stack(embedding_lists.values)

    # --- 步驟 1: PCA (768D -> 50D) ---
    # 這是標準SOP，先用 PCA 快速降維，能大幅加速 t-SNE
    print("步驟 1/2：執行 PCA (768D -> 50D)...")
    pca = PCA(n_components=50)
    vectors_50d = pca.fit_transform(vectors_768d)
    print("PCA 執行完畢。")

    # --- 步驟 2: t-SNE (50D -> 3D) ---
    # 這是最耗時的步驟，但效果最好
    print("步驟 2/2：執行 t-SNE (50D -> 3D)... (這可能需要幾分鐘，請稍候)")
    
    tsne = TSNE(
        n_components=3,  # 我們要降到 3 維
        perplexity=30,   # 一個標準的 t-SNE 參數
        #n_iter=1000,      # 迭代 1000 次
        init='pca',      # 使用 PCA 結果初始化
        learning_rate='auto',
        verbose=1        # 印出進度
    )
    
    start_time = time.time()
    vectors_3d = tsne.fit_transform(vectors_50d)
    end_time = time.time()
    
    print(f"t-SNE 執行完畢。花費時間：{ (end_time - start_time):.2f} 秒。")
    
    return vectors_3d

def plot_3d(df, vectors_3d):
    """使用 Plotly 繪製可互動的 3D 散點圖"""
    if vectors_3d is None:
        print("沒有 3D 向量可供繪圖。")
        return

    print(f"正在生成 3D 互動式圖表...")
    
    # --- [關鍵] 設定視覺化 ---
    # 我們將每個點的「顏色」設為它的「價格」
    # 我們將每個點的「標籤」設為它的「品牌」
    hover_text = [
        f"Brand: {brand}<br>Price: ${price:.2f}" 
        for brand, price in zip(df['brand'], df['sales_price'])
    ]

    fig = go.Figure(data=[go.Scatter3d(
        x=vectors_3d[:, 0],  # 3D 座標的 x 軸
        y=vectors_3d[:, 1],  # 3D 座標的 y 軸
        z=vectors_3d[:, 2],  # 3D 座標的 z 軸
        mode='markers',
        
        # 當滑鼠懸停時，顯示的文字 (品牌 + 價格)
        text=hover_text, # 使用我們預先生成的文字
        hoverinfo='text',
        
        marker=dict(
            size=4,                # 點的大小
            color=df['sales_price'], # 【關鍵】用「價格」來決定顏色
            colorscale='Viridis',  # 使用 'Viridis' 色階
            colorbar_title='Price ($)', # 顏色條的標題
            opacity=0.8
        )
    )])

    fig.update_layout(
        title=f"768D 向量降維至 3D 視覺化 (t-SNE, n={len(df)})",
        scene=dict(
            xaxis_title='t-SNE Component 1',
            yaxis_title='t-SNE Component 2',
            zaxis_title='t-SNE Component 3'
        ),
        margin=dict(r=20, b=10, l=10, t=40) # 調整邊界
    )

    # --- [關鍵] 儲存為 HTML 檔案 ---
    try:
        fig.write_html(OUTPUT_HTML_FILE)
        print("\n" + "=" * 40)
        print("【成功！】")
        print(f"已生成互動式 3D 圖表： {OUTPUT_HTML_FILE}")
        print(f"請您直接將這個 '{OUTPUT_HTML_FILE}' 檔案拖曳到您的 Chrome 或 Firefox 瀏覽器中開啟。")
        print("=" * 40)
    except Exception as e:
        print(f"儲存圖表時發生錯誤：{e}")

# --- 主程式 ---
if __name__ == "__main__":
    
    # 1. 讀取資料
    df_data = fetch_data_from_db()
    
    if df_data is not None and not df_data.empty:
        # 2. 降維
        vectors_3d = reduce_dimensions(df_data)
        
        # 3. 繪圖
        plot_3d(df_data, vectors_3d)
    else:
        print("無法執行，沒有從資料庫讀取到任何資料。")

正在連線至資料庫 'final_project'...
讀取資料庫時發生錯誤：connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

無法執行，沒有從資料庫讀取到任何資料。


# 建立資料表格

import psycopg2
from psycopg2 import sql
import os
from dotenv import load_dotenv

# --- 載入 .env 檔案 ---
load_dotenv()

# --- 資料庫連線設定 ---
DB_SETTINGS = {
    "host": os.environ.get("DB_HOST"),
    "port": os.environ.get("DB_PORT"),
    "user": os.environ.get("DB_USER"),
    "password": os.environ.get("DB_PASSWORD"),
    "database": os.environ.get("DB_NAME")
}
